In [1]:
#! pip install pixivpy

In [1]:
def get_illusts(date):
    illusts = []
    json_result = api.illust_ranking('day', date=date)
    if json_result is None: return illusts
    if json_result.illusts is None: return illusts
    illusts.extend(json_result.illusts)
    while True:
        if json_result.next_url is None: return illusts
        next_qs = api.parse_qs(json_result.next_url)
        if next_qs is None: return illusts
        json_result = api.illust_ranking(**next_qs)
        if json_result is None: return illusts
        if json_result.illusts is None: return illusts
        illusts.extend(json_result.illusts)

In [2]:
from pixivpy3 import *
from datetime import date, timedelta

refresh_token = 'GwWqT83JBSi4M_mBqs6B6za5cH9xeq0gRlI69zYlfvk' 

api = AppPixivAPI()
api.auth(refresh_token=refresh_token) # api.login("username", "password")   # Not required

{'access_token': 'BjcgwrcVT6Xee7iOzfxYSMhYvm1V7YSkac-qd73ewlY',
 'expires_in': 3600,
 'token_type': 'bearer',
 'scope': '',
 'refresh_token': 'GwWqT83JBSi4M_mBqs6B6za5cH9xeq0gRlI69zYlfvk',
 'user': {'profile_image_urls': {'px_16x16': 'https://s.pximg.net/common/images/no_profile_ss.png',
   'px_50x50': 'https://s.pximg.net/common/images/no_profile_s.png',
   'px_170x170': 'https://s.pximg.net/common/images/no_profile.png'},
  'id': '64763103',
  'name': 'eyesssewnshut',
  'account': 'user_chrm5557',
  'mail_address': 'followdemiurg@gmail.com',
  'is_premium': False,
  'x_restrict': 1,
  'is_mail_authorized': True},
 'response': {'access_token': 'BjcgwrcVT6Xee7iOzfxYSMhYvm1V7YSkac-qd73ewlY',
  'expires_in': 3600,
  'token_type': 'bearer',
  'scope': '',
  'refresh_token': 'GwWqT83JBSi4M_mBqs6B6za5cH9xeq0gRlI69zYlfvk',
  'user': {'profile_image_urls': {'px_16x16': 'https://s.pximg.net/common/images/no_profile_ss.png',
    'px_50x50': 'https://s.pximg.net/common/images/no_profile_s.png',


In [3]:
good_images = [] 

for i in range(150):
    curr_date = str(date.today() - timedelta(3 * i  + 1))
    illusts   = get_illusts(curr_date)
    for illust in illusts:
        if illust.type == 'manga': continue
        good_images.append(illust.id)

In [4]:
good_images = set(good_images)

In [5]:
import pickle

with open('good_images.pickle', 'wb') as handle:
    pickle.dump(good_images, handle, pickle.HIGHEST_PROTOCOL)

In [3]:
import re

def preprocess_tag(tag):
    
    tag = tag.lower()
    tag = re.sub(r'\s\d+\+\s(bookmarks)', ' ', tag)
    tag = tag.strip()
    
    # TODO: add more preprocessing?
    
    return tag

In [4]:
import pickle

with open('good_images.pickle', 'rb') as handle: good_img = pickle.load(handle)

In [26]:
popular_tags = []

for i in range(150):
    curr_date = str(date.today() - timedelta(3 * i  + 1))
    illusts   = get_illusts(curr_date)
    for illust in illusts:
        
        if illust.type == 'manga': continue
        if not illust.id in good_img: continue
            
        tags = [preprocess_tag(t['translated_name']) 
                for t in illust.tags if t['translated_name'] is not None]
        tags = list(set(tags))
        
        popular_tags.extend(tags)
        good_img.remove(illust.id)
        

In [27]:
from collections import Counter

popular_tags = dict(Counter(popular_tags))
popular_tags = {k: v for k, v in popular_tags.items() if v >= 20}
popular_tags = {k: v for k, v in sorted(popular_tags.items(), key = lambda x: x[1], reverse=True)}

In [28]:
with open('popular_tags.pickle', 'wb') as handle:
    pickle.dump(popular_tags, handle, pickle.HIGHEST_PROTOCOL)

In [5]:
with open('popular_tags.pickle', 'rb') as handle: popular_tags = pickle.load(handle)

In [8]:
len(good_img)

33630

In [6]:
! mkdir images_pixiv

In [9]:
import os
from tqdm.notebook import tqdm

map_files_to_tags = {}

for i in tqdm(range(100, 150), desc='downloading stuff'):
    curr_date = str(date.today() - timedelta(3 * i  + 1))
    illusts   = get_illusts(curr_date)
    for illust in illusts:
        
        if illust.type == 'manga': continue
        if not illust.id in good_img: continue
            
        tags = [preprocess_tag(t['translated_name']) 
                for t in illust.tags if t['translated_name'] is not None]
        tags = list(set(tags))
        
        counter = 0
        for t in tags: 
            if t in popular_tags: counter+= popular_tags[t]
                
        if counter < 100: continue
            
        extension = str(illust.image_urls['medium']).split('.')[-1]
        filename  = str(illust.id) + extension
        
        map_files_to_tags[filename] = tags
        
        api.download(
            illust.image_urls['medium'], 
            path=os.path.join(os.getcwd(), 'images_pixiv'),
            name=filename
        )
        
        good_img.remove(illust.id)

downloading stuff:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
#!rm -r images_pixiv

In [10]:
with open('files_to_tags_3.pickle', 'wb') as handle:
    pickle.dump(map_files_to_tags, handle, pickle.HIGHEST_PROTOCOL)

In [11]:
map_files_to_tags

{'81363132jpg': ['kanao tsuyuri',
  'demon slayer: kimetsu no yaiba',
  'demon slayer',
  'kocho sisters',
  'copyright'],
 '81377438jpg': ['fate/grand order', 'fate/go'],
 '81370476jpg': ['leotard', 'sheer skirt', 'original'],
 '81371037jpg': ['sweet',
  'original',
  'nail polish',
  'i like it when you eat a lot',
  'girl',
  'original 7,500+ bookmarks'],
 '81365288jpg': ['created by the original author',
  'mikasa (azur lane)',
  'japanese wedding dress',
  'azur lane'],
 '81372309jpg': ['ozymandias (fate)',
  'gilgamesh (fate)',
  'crypter (fate)',
  'fate/go',
  'fate/grand order',
  'knights of the round table'],
 '81363134jpg': ['pokémon',
  'volcarona',
  'dynamax',
  'pocket monsters',
  'mothra'],
 '81365904jpg': ['virtual youtuber', 'nijisanji', 'ange katrina'],
 '81372816jpg': ['original',
  'ganbare, douki-chan',
  "feels like we can't win",
  "women's battle"],
 '81363927jpg': ['girl', 'young girl'],
 '81375324jpg': ['lingerie',
  'arknights',
  'schwarz (arknights)',
  

In [1]:
import pickle
import os

In [4]:
with open('files_to_tags_1.pickle', 'rb') as handle:
    map_files_to_tags0 = pickle.load(handle)

with open('files_to_tags_2.pickle', 'rb') as handle:
    map_files_to_tags1 = pickle.load(handle)
    
with open('files_to_tags_3.pickle', 'rb') as handle:
    map_files_to_tags2 = pickle.load(handle)

In [8]:
map_files_to_tags = {
    k:v for k, v in \
    list(map_files_to_tags0.items()) +\
    list(map_files_to_tags1.items()) +\
    list(map_files_to_tags2.items())
}

In [25]:
from PIL import Image
import os
import numpy as np

In [46]:
good_images = {}


for k, v in map_files_to_tags.items():
    
    img = Image.open(os.path.join(os.getcwd(), f'images_pixiv/{k}'))
    if len(img.getbands()) == 1: continue 
        
    h, w = img.size
    if not h % 2 == 0: h += 1
    if not w % 2 == 0: w += 1
        
    
    max_ = max(h, w)
    min_ = min(h, w)
    
    ratio = float(max_) / float(min_)
    if ratio >= 2: continue
        
    img = img.resize((h, w), Image.ANTIALIAS)    
    
    if h != w:
        img = np.array(img)
        pad_img = np.ones((max_, max_, 3)) * 255
        if h == max_:
            pad_img[(h-w)//2:-(h-w)//2] = img
        else:
            pad_img[:, (w-h)//2:-(w-h)//2] = img
        img = Image.fromarray(pad_img.astype(np.uint8))
        
    img = img.resize((256, 256), Image.ANTIALIAS)
    #img.show()
    
    good_images[f'{k[:-3]}.{k[-3:]}'] = v   
    img.save(os.path.join(os.getcwd(), f'images_256/{k[:-3]}.{k[-3:]}'))

    

In [34]:
! mkdir images_256